In [ ]:
# Python version
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
import matplotlib.pyplot as plt
# pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))
import pickle as pickle

In [ ]:
def pickle_data(filename, data):

    fileObject = open('PickleData/'+filename,'wb') 
    # this writes the object a to the
    # file named 'testfile'
    pickle.dump(data,fileObject) 

    # here we close the fileObject
    fileObject.close()
    
def load_pickle_data(filename):
    # we open the file for reading
    fileObject = open('PickleData/'+filename,'rb')  
    # load the object from the file into var b
    return pickle.load(fileObject)

In [ ]:
#Load validation set predictions from logistic regr model on binary dataset
pred_y = load_pickle_data('test_y_opt_xgboost')

In [ ]:
#Load training Dataset
df_train = pd.read_csv('Data/train.csv')

In [ ]:
df_test = pd.read_csv('Data/test.csv')

In [ ]:
# base_bid = sum(df_train['payprice'])/df_train.shape[0]
# OR BAse_bid is optimised value
base_bid = 30
avg_ctr = df_train[df_train['click']==1].shape[0]/df_train.shape[0]*100
p_ctr = pred_y

In [ ]:
#Using formula froma ssignment decription
bid = base_bid * p_ctr/avg_ctr

In [ ]:
threshold = 0.5
#New col to store prediction probabilty from model
df_test['pred_y'] = pred_y
#Extract all imps with prediction probability over threshold
bid_imps = df_test[df_test['pred_y']>=threshold]

In [ ]:
#sort in descending order of probabilty
bid_imps = bid_imps.sort_values(by=['pred_y'], ascending=False)

In [ ]:
budget = 6250000

budget_list = []

def simulate_bids(data):
    
    # Running budget
    budget_spent = 0
    
    index = 0

    for index, row in data.iterrows():

        #Budget exceeded
        if (budget_spent >= budget):
            break

        #Update budget
        #Using formula from assignment decription
        bid = base_bid * row['pred_y']/avg_ctr
        
        budget_spent = budget_spent + bid
        data.loc[index, 'bidprice'] = bid
        print('Bid Price:',bid)
        print('Budget Spent:',budget_spent)
        print('----------------')
        
    print('Budget Spent:',budget_spent)
    budget_list.append(budget_spent)
    return data

In [ ]:
bid_imps = simulate_bids(bid_imps)

In [ ]:
#Sort back into index position
bid_imps.sort_index(inplace=True)

In [ ]:
#Merge bid column with original dataframe
df_test = pd.concat([df_test, bid_imps['bidprice']], axis=1)

In [ ]:
#change NaNs to 0
df_test['bidprice'].fillna(0, inplace=True)

In [ ]:
df_test_submittion = df_test[['bidid','bidprice']]

In [ ]:
# Cast all values to integer
df_test_submittion['bidprice'] = df_test_submittion['bidprice'].astype(int)

In [ ]:
df_test_submittion

In [ ]:
df_test_submittion.to_csv('Submissions/XGBoost_test_optmised.csv')

##### For Testing Purposes

In [ ]:
def bid_on_impressions(threshold):
    df_test['pred_y'] == pred_y
    bid_imps = df_test[df['pred_y']>=threshold]

In [ ]:
def calculate_winning_bids(data):
    
    success_bid = data.loc[data['predclick']  & (data['predbid'] > data['payprice'])]
    success_bid = success_bid.reset_index(drop=True)
    
    return success_bid




In [ ]:
successfull_bids = calculate_winning_bids(df_val)

In [ ]:
click_const_bid = sum(successfull_bids.iloc[0:calculate_budget(successfull_bids),:]['click'])

In [ ]:
print('Impression with clicks won:',click_const_bid)